# Sentiment Analysis

In [1]:
import pandas as pd

splits = {'train': 'train_df.csv', 'validation': 'val_df.csv', 'test': 'test_df.csv'}
dataframes = []
for split, filename in splits.items():
    if filename != 'val_df.csv':
      df = pd.read_csv(f'hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/{filename}')
      df.drop(['id', 'sentiment'], axis=1, inplace=True)
      dataframes.append(df)
    else:
      test_df = pd.read_csv(f'hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/{filename}')
      test_df.drop(['id', 'sentiment'], axis=1, inplace=True)
train_df = pd.concat(dataframes)
train_df = train_df.dropna()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
print(train_df.head())
print(test_df.head())

print(len(train_df))
print(len(test_df))

                                                text  label
0                    Cooking microwave pizzas, yummy      2
1  Any plans of allowing sub tasks to show up in ...      1
2   I love the humor, I just reworded it. Like sa...      2
3                       naw idk what ur talkin about      1
4          That sucks to hear. I hate days like that      0
                                                text  label
0  Laying in bed til workkk... Oh the life. Defin...      0
1   ooohhh imma need you to get on that asap love...      2
2   Thanks! I love it they have a video, so you d...      2
3     I left my ipod in the car so now its all warm.      2
4  Great app. Only complaint is that I'd like the...      2
36437
5205


## Train the Model

In [3]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=34b355f41e508993f696b4296704d93ae7a2451887f2b34e82da86c25b91a05c
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs()

model_args.overwrite_output_dir=True
model_args.evaluate_during_training=True
model_args.no_save = True

model_args.max_seq_length=256
model_args.use_early_stopping=True
model_args.early_stopping_delta=0.01
model_args.early_stopping_metric='eval_loss'
model_args.early_stopping_metric_minimize=True
model_args.early_stopping_patience=2
model_args.evaluate_during_training_steps=32
model_args.reprocess_input_data=True
model_args.manual_seed=4
model_args.use_multiprocessing=True
model_args.labels_list=[0, 1, 2]
model_args.wandb_project="Sentiment Analysis Sweep"

In [5]:
import wandb

def train_model():
  wandb.init()
  model = ClassificationModel("roberta", "roberta-base", num_labels=3, args=model_args, use_cuda=True, sweep_config=wandb.config)
  model.train_model(train_df, eval_df=test_df)

In [11]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [12]:
wandb.login()

sweep_configuration = {
	"method": "grid",
	"metric": {"goal": "minimize", "name": "eval_loss"},
	"parameters": {
		"train_epochs": {"values": [8, 10, 12, 16]},
		"train_batch_size": {"values": [16, 32, 64]},
		"learning_rate": {"values": [1e-6, 5e-5, 1e-5, 5e-4]},
	}
}
sweep_id = input("Please enter an existing sweep id if you want to continue an existing sweep (leave blank for a new sweep): ")
sweep_id = None if sweep_id == "" else sweep_id

if sweep_id is None:
	sweep_id = wandb.sweep(sweep=sweep_configuration, project="Sentiment Analysis Sweep")

wandb.agent(sweep_id, function=train_model)

Please enter an existing sweep id if you want to continue an existing sweep (leave blank for a new sweep): 
Create sweep with ID: veq20kar
Sweep URL: https://wandb.ai/d3dsec/Sentiment%20Analysis%20Sweep/sweeps/veq20kar


wandb: Agent Starting Run: 4uwnnufm with config:
wandb: 	learning_rate: 1e-06
wandb: 	train_batch_size: 16
wandb: 	train_epochs: 8
wandb: Currently logged in as: simonas-mickus (d3dsec) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
wandb: Ctrl + C detected. Stopping sweep.
